In [2]:
import numpy as np
from scrfd import SCRFD

In [3]:
model_path = "./models/scrfd.onnx"
model = SCRFD(model_path)

/home/htsc/nguyentd/face_detection/scrfd/venv/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [21]:
height = 640 // 8
width = 640 // 8

anchor_centers = np.stack(
    np.mgrid[:height, :width][::-1],  # type: ignore
    axis=-1,
).astype(np.float32)

anchor_centers = (anchor_centers * 8).reshape((-1, 2))
print(anchor_centers[:5])
anchor_centers = np.stack([anchor_centers] * 2, axis=1).reshape((-1, 2))
print(anchor_centers[:5])
print(anchor_centers.shape)

[[ 0.  0.]
 [ 8.  0.]
 [16.  0.]
 [24.  0.]
 [32.  0.]]
[[ 0.  0.]
 [ 0.  0.]
 [ 8.  0.]
 [ 8.  0.]
 [16.  0.]]
(12800, 2)


In [23]:
feature_height = 640 // 8
feature_width = 640 // 8

anchor_centers = np.mgrid[:feature_height, :feature_width]  # 0 to h, 0 to w
anchor_centers = anchor_centers[::-1]  # 0 to w, 0 to h
anchor_centers = anchor_centers.transpose(1, 2, 0)  # shape (h, w, 2)
anchor_centers = anchor_centers.reshape(-1, 2)  # shape (h*w, 2)
anchor_centers *= 8

print(anchor_centers[:5])
anchor_centers = np.hstack([anchor_centers] * 2).reshape((-1, 2))
print(anchor_centers[:5])
print(anchor_centers.shape)

[[ 0  0]
 [ 8  0]
 [16  0]
 [24  0]
 [32  0]]
[[ 0  0]
 [ 0  0]
 [ 8  0]
 [ 8  0]
 [16  0]]
(12800, 2)


In [24]:
np.maximum(5, np.arange(10))

array([5, 5, 5, 5, 5, 5, 6, 7, 8, 9])